<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C4_Spark_RDD_Dataframe_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# Basic Data Analysis | Supermarket | Netflix | IMDB
Demonstrate the usage of RDD, Spark Dataframe and Spark SQL to explore the same datafile. <br>
More detailed tutorial available at https://github.com/Praxis-QR/BDSN/blob/main/Complex_Complete_Spark_Tutorial.ipynb
<hr>


In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))

!python --version
!lsb_release -a

#check which version of MongoDB  is available
#!apt-cache policy mongodb

#check which versions of software are available
#!pip3 index versions pyspark
#!pip3 index versions pyngrok

ॐ श्री सरस्वत्यै नमः 2024-04-29 09:01:11.774805+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


# Download Sample Data as CSV files
Sourced from https://github.com/search?q=owner%3Ajigsawlabs-student%20pyspark&type=repositories

In [2]:
import pandas as pd
import re

In [3]:
#!wget -O netflix.csv "https://raw.githubusercontent.com/jigsawlabs-student/pyspark-cluster-lab/main/netflix_titles.csv"
!wget -q "https://github.com/Praxis-QR/BDSN/raw/main/Data2/NetFlix.csv"
#df = pd.read_csv('s3://jigsaw-labs-student/supermarket_sales.csv')
df1 = pd.read_csv('NetFlix.csv')

!wget -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/SuperMarketSales.csv'
df2 = pd.read_csv('SuperMarketSales.csv')

#df2 = pd.read_csv("s3://jigsaw-labs-student/imdb_movies.csv")
!wget -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/IMDBmovies.csv'
df3 = pd.read_csv('IMDBmovies.csv')
df1.shape, df2.shape, df3.shape

((7787, 12), (1000, 17), (2000, 7))

# Spark Installation


## Full Installation - Skip, not required

In [ ]:
#!apt update > /dev/null
#!apt install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Get latest and correct version of Spark
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
#!pip install -q pyspark

#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


In [ ]:
# findspark route is no more required
#import findspark
#findspark.init()
#
#findspark.init('/content/spark-2.2.2-bin-hadoop2.7')
#
# https://stackoverflow.com/questions/42223498/findspark-init-indexerror-list-index-out-of-range-error
#


## Direct Install with pip

In [4]:
!pip3 -q install pyspark
#
#from pyspark.sql.functions import col, to_date, to_timestamp
#from pyspark.sql.functions import regexp_replace
#from pyspark.sql.types import StructType,StructField, StringType, DateType, TimestampType, IntegerType,DoubleType, FloatType,NumericType, DecimalType
#
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
#
spark39 = SparkSession.builder.master("local[*]").\
            config('spark.ui.port', '4050').\
            appName("Yantrajaal").\
            getOrCreate()
#
sc = spark39.sparkContext
#
# The spark Console UI is available in the link that will be displayed in this cell.
# If you do not wish to use the Console, you may skip the Tunnel part
sc

<SparkContext master=local[*] appName=Yantrajaal>

# Supermarket Sales Data

In [6]:
!head SuperMarketSales.csv

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3
699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,3/25/2019,18:30,Ewallet,597.73,4.761904762,29.8865,4.1
355-53-5943,A,Yangon,Member,Female,Electronic accesso

## RDD Operations

### Read CSV into RDD

In [7]:
sales_RDD1 = sc.textFile("SuperMarketSales.csv")
sales_RDD2 = spark39.read.text("SuperMarketSales.csv").rdd
# for an explanation of the alternate way to read text files
# see this https://stackoverflow.com/questions/52665353/reading-a-text-file-in-spark/52669632#52669632

In [8]:
sales_RDD1.take(5)

['Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating',
 '750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1',
 '226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6',
 '631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4',
 '123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4']

In [9]:
sales_RDD2.take(5)

[Row(value='Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating'),
 Row(value='750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1'),
 Row(value='226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6'),
 Row(value='631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4'),
 Row(value='123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4')]

In [10]:
# Delete the first, header row
#
!sed '1d' SuperMarketSales.csv > Headless_SuperMarketSales.csv
#!sed -i '1d' SuperMarketSales.csv
sales_RDD1 = sc.textFile("Headless_SuperMarketSales.csv")
sales_RDD2 = spark39.read.text("Headless_SuperMarketSales.csv").rdd
sales_RDD1.take(5)
#sales_RDD2.take(5)

['750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1',
 '226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6',
 '631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4',
 '123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4',
 '373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3']

In [11]:
sales_RDD2.take(5)

[Row(value='750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1'),
 Row(value='226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6'),
 Row(value='631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4'),
 Row(value='123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4'),
 Row(value='373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3')]

### Data Analysis with RDD

In [12]:
# Total Number of Rows
sales_RDD1.count()

1000

#### Filter Data

In [13]:
# Filter, or segregate the data, based on payment method
#
sales_wallet = sales_RDD1.filter(lambda x: ",Ewallet," in x)
sales_cash = sales_RDD1.filter(lambda x: ",Cash," in x)
sales_card = sales_RDD1.filter(lambda x: ",Credit card," in x)
#
wallet_count = sales_wallet.count()
cash_count = sales_cash.count()
card_count = sales_card.count()
print(wallet_count, cash_count, card_count)

345 344 311


#### Set Operations

In [14]:
# Another way to segregate the data
#
sales_wallet = sales_RDD1.filter(lambda x: ",Ewallet," in x)
wallet_count = sales_wallet.count()
#
sales_non_wallet = sales_RDD1.subtract(sales_wallet)
non_wallet_count = sales_non_wallet.count()
print(wallet_count, non_wallet_count)

345 655


In [15]:
# Get Unique Values of field City
#
sales_RDD_csv = sales_RDD1.map(lambda x: x.split(","))
cities = sales_RDD_csv.map(lambda x: x[2]).distinct()
cities.collect()

['Naypyitaw', 'Yangon', 'Mandalay']

In [16]:
sales_RDD_csv = sales_RDD1.map(lambda x: x.split(","))
product_lines = sales_RDD_csv.map(lambda x: x[5]).distinct()
product_lines.collect()

['Health and beauty',
 'Home and lifestyle',
 'Electronic accessories',
 'Sports and travel',
 'Food and beverages',
 'Fashion accessories']

#### Aggregation
Calculating Count, Sum, Average etc

In [17]:
# Element 7 in the dataset is Quantity
# Convert str to int
#
def x4m1(text):
    elements = text.split(",")
    return(int(elements[7]))

In [18]:
# Extract Quantity from dataset
#
sales_wallet_Qty = sales_wallet.map(lambda x: x4m1(x))
sales_wallet_Qty.take(3)

[7, 8, 7]

In [19]:
# Calculate sum of all Quantity
#
wallet_total_Qty = sales_wallet_Qty.reduce(lambda x,y : x+y)
print('wallet total quantity = ',wallet_total_Qty, '\nwallet average quantity =', wallet_total_Qty/wallet_count)

wallet total quantity =  1892 
wallet average quantity = 5.484057971014493


#### Accumulators
Reduce the number of passes through the data.<br>
For an explanation of the aggregate function see [this stackoverflow post](https://stackoverflow.com/questions/28240706/explain-the-aggregate-functionality-in-spark-with-python-and-scala/38949457#38949457).

In [20]:
wallet_data = sales_wallet_Qty.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
    )

print('Wallet Quantity Total',wallet_data[0],\
      '\nWallet Quantity Count',wallet_data[1],\
      '\nWallet Quantity Average', wallet_data[0]/wallet_data[1])

Wallet Quantity Total 1892 
Wallet Quantity Count 345 
Wallet Quantity Average 5.484057971014493


#### Aggregation on Multiple (two) Data Elements

In [21]:
# Element 7 in the dataset is Quantity
# Element 15 is Gross Income
#
def x4m2(text):
    elements = text.split(",")
    return(int(elements[7]), float(elements[15]))

In [22]:
sales_wallet_Qty_Inc = sales_wallet.map(lambda x: x4m2(x))
sales_wallet_Qty_Inc.take(3)

[(7, 26.1415), (8, 23.288), (7, 30.2085)]

In [23]:
wallet_data = sales_wallet_Qty_Inc.aggregate(
    (0,0,0), # the initial value
    (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)), # combine value with acc
    (lambda acc1, acc2 : (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])) # combine accumulators
    )

print('Wallet Quantity Total',wallet_data[0],\
      '\nWallet Income Total',wallet_data[1],\
      '\nWallet Count',wallet_data[2],\
      '\nWallet Quantity Average',wallet_data[0]/wallet_data[2],\
      '\nWallet Income Average', wallet_data[1]/wallet_data[2]\
      )
#print('Wallet Quantity Average = ', wallet_data[0]/wallet_data[2],'\nWallet Gross Income Average = ', wallet_data[1]/wallet_data[2])

Wallet Quantity Total 1892 
Wallet Income Total 5237.767 
Wallet Count 345 
Wallet Quantity Average 5.484057971014493 
Wallet Income Average 15.181933333333333


#### Chained Operations
Eliminate intermediate RDD creation

In [24]:
wallet_data = sc.textFile("Headless_SuperMarketSales.csv").\
    filter(lambda x: ",Ewallet," in x).\
    map(lambda x: x4m2(x)).\
    aggregate(                                                                      # Perform Aggregation on multiple fields
    (0,0,0),                                                                        # the initial value
    (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)),        # combine value with acc
    (lambda acc1, acc2 : (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])) # combine accumulators
    )

print('Wallet Quantity Total',wallet_data[0],\
      '\nWallet Income Total',wallet_data[1],\
      '\nWallet Count',wallet_data[2],\
      '\nWallet Quantity Average',wallet_data[0]/wallet_data[2],\
      '\nWallet Income Average', wallet_data[1]/wallet_data[2]\
      )

#print(wallet_data[0],wallet_data[1],wallet_data[2])
#print('Wallet Quantity Average = ', wallet_data[0]/wallet_data[2],'\nWallet Gross Income Average = ', wallet_data[1]/wallet_data[2])

Wallet Quantity Total 1892 
Wallet Income Total 5237.767 
Wallet Count 345 
Wallet Quantity Average 5.484057971014493 
Wallet Income Average 15.181933333333333


In [25]:
# For all three type of payment options
#
for pType in ['Ewallet','Cash','Credit card']:
    _pType = ","+pType+","
    payment_data = sc.textFile("Headless_SuperMarketSales.csv").\
        filter(lambda x: _pType in x).\
        map(lambda x: x4m2(x)).\
        aggregate(                                                                      # Perform Aggregation on multiple fields
        (0,0,0),                                                                        # the initial value
        (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)),        # combine value with acc
        (lambda acc1, acc2 : (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])) # combine accumulators
        )

    print('Payment Type', pType,\
          '\nQuantity Total',payment_data[0],\
        'Income Total',payment_data[1],\
        'Count',payment_data[2],\
        '\nQuantity Average',payment_data[0]/payment_data[2],\
        'Income Average', payment_data[1]/payment_data[2],\
        '\n............................................',\
        )


Payment Type Ewallet 
Quantity Total 1892 Income Total 5237.767 Count 345 
Quantity Average 5.484057971014493 Income Average 15.181933333333333 
............................................
Payment Type Cash 
Quantity Total 1896 Income Total 5343.170000000001 Count 344 
Quantity Average 5.511627906976744 Income Average 15.532470930232561 
............................................
Payment Type Credit card 
Quantity Total 1722 Income Total 4798.432000000001 Count 311 
Quantity Average 5.536977491961415 Income Average 15.42904180064309 
............................................


In [26]:
# For all three type of payment options
# HERE WE READ the file just ONCE
#
rdd1 = sc.textFile("Headless_SuperMarketSales.csv")
for pType in ['Ewallet','Cash','Credit card']:
    _pType = ","+pType+","
    payment_data = rdd1.\
        filter(lambda x: _pType in x).\
        map(lambda x: x4m2(x)).\
        aggregate(                                                                      # Perform Aggregation on multiple fields
        (0,0,0),                                                                        # the initial value
        (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)),        # combine value with acc
        (lambda acc1, acc2 : (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])) # combine accumulators
        )

    print('Payment Type', pType,\
          '\nQuantity Total',payment_data[0],\
        'Income Total',payment_data[1],\
        'Count',payment_data[2],\
        '\nQuantity Average',payment_data[0]/payment_data[2],\
        'Income Average', payment_data[1]/payment_data[2],\
        '\n............................................',\
        )


Payment Type Ewallet 
Quantity Total 1892 Income Total 5237.767 Count 345 
Quantity Average 5.484057971014493 Income Average 15.181933333333333 
............................................
Payment Type Cash 
Quantity Total 1896 Income Total 5343.170000000001 Count 344 
Quantity Average 5.511627906976744 Income Average 15.532470930232561 
............................................
Payment Type Credit card 
Quantity Total 1722 Income Total 4798.432000000001 Count 311 
Quantity Average 5.536977491961415 Income Average 15.42904180064309 
............................................


### Convert RDD to Spark DataFrame

In [27]:
# Split string data on "," and convert string to relevant datatypes
#
#sales_RDD1_split1 = sales_RDD1.map(lambda l: l.split(","))
#sales_RDD1_split1.take(1)
#sales_RDD1_split2 = sales_RDD1_split1.map(lambda l: [l[0],l[1],l[2],l[3],l[4],l[5],float(l[6]),int(l[7]),float(l[8]),float(l[9]),l[10],l[11],l[12],float(l[13]),float(l[14]),float(l[15]),float(l[16])])
#sales_RDD1_split2.take(1)

def x4m3(text):
    l = text.split(",")
    return([l[0],l[1],l[2],l[3],l[4],l[5],float(l[6]),int(l[7]),float(l[8]),float(l[9]),l[10],l[11],l[12],float(l[13]),float(l[14]),float(l[15]),float(l[16])])

sales_RDD1_csv = sales_RDD1.map(lambda x: x4m3(x))
#sales_RDD1_split.take(1)

In [28]:
# Convert to dataframe
# Date, Time does not convert correctly
#
SalesDataSchema = StructType([
    StructField('Invoice ID', StringType(), True),
    StructField('Branch', StringType(), True),
    StructField('City', StringType(), True),
    StructField('Customer Type', StringType(), True),
    StructField('Gender', StringType(), True),
    StructField('Product Line', StringType(), True),
    StructField('Unit Price', FloatType(), True),
    StructField('Quantity', IntegerType(), True),
    StructField('Tax 5%', DoubleType(), True),
    StructField('Total', DoubleType(), True),
    StructField('Date', StringType(), True),
    StructField('Time', StringType(), True),
    StructField('Payment', StringType(), True),
    StructField('cogs', DoubleType(), True),
    StructField('gross margin percentage', DoubleType(), True),
    StructField('gross income', DoubleType(), True),
    StructField('Rating', DoubleType(), True)
])

#sales_df1 = spark39.createDataFrame(data = sales_RDD1_split2, schema = colSchema)
sales_df1 = spark39.createDataFrame(data = sales_RDD1_csv, schema = SalesDataSchema)

#sales_df1.printSchema()
#sales_df1.show(5)

In [29]:
# Fix Date, Time datatype
#
#sales_df1 = sales_df1.withColumn('Date',to_date(col('Date'), "M/d/y"))
#sales_df1 = sales_df1.withColumn('Time',to_timestamp(col('Time'), "HH:mm"))

sales_df1 = sales_df1.\
    withColumn('Date',to_date(col('Date'), "M/d/y")).\
    withColumn('Time',to_timestamp(col('Time'), "HH:mm"))

#sales_df1 = sales_df1.

sales_df1.printSchema()
sales_df1.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product Line: string (nullable = true)
 |-- Unit Price: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer Type|Gender|        Product Line|Unit Price|Quantity| Tax 5%|   To

## Dataframe Operations

### Read CSV into Spark Dataframe

In [ ]:
# Relaod Supermarket Sales Data WITH HEADERS
# Original File has had its header line deleted
#
#!wget -O SuperMarketSales.csv -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/SuperMarketSales.csv'

In [30]:
# Date is not inferred correctly
# Needs to be fixed in next step
#
sales_df2 = spark39.read.format("csv").option("header", "true").option("inferSchema", "true").load("./SuperMarketSales.csv")
#sales_df2.printSchema()
#sales_df2.show(5)

In [31]:
sales_df2 = sales_df2.withColumn('Date',to_date(col('Date'), "M/d/y"))
sales_df2.printSchema()
sales_df2.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   T

### Display a subset of columns

In [32]:
# All three works
#
#sales_df2.select(sales_df2['Invoice ID'], sales_df2['gross income'],sales_df2['City'],sales_df2['Payment']).show(truncate=False)
#sales_df2.select(['Invoice ID','gross income','City','Payment']).show(truncate=False)
sales_df2['Invoice ID','gross income','City','Payment'].show(truncate=False)

+-----------+------------+---------+-----------+
|Invoice ID |gross income|City     |Payment    |
+-----------+------------+---------+-----------+
|750-67-8428|26.1415     |Yangon   |Ewallet    |
|226-31-3081|3.82        |Naypyitaw|Cash       |
|631-41-3108|16.2155     |Yangon   |Credit card|
|123-19-1176|23.288      |Yangon   |Ewallet    |
|373-73-7910|30.2085     |Yangon   |Ewallet    |
|699-14-3026|29.8865     |Naypyitaw|Ewallet    |
|355-53-5943|20.652      |Yangon   |Ewallet    |
|315-22-5665|36.78       |Naypyitaw|Ewallet    |
|665-32-9167|3.626       |Yangon   |Credit card|
|692-92-5582|8.226       |Mandalay |Credit card|
|351-62-0822|2.896       |Mandalay |Ewallet    |
|529-56-3974|5.102       |Mandalay |Cash       |
|365-64-0515|11.7375     |Yangon   |Ewallet    |
|252-56-2699|21.595      |Yangon   |Ewallet    |
|829-34-3910|35.69       |Yangon   |Cash       |
|299-46-1805|28.116      |Mandalay |Cash       |
|656-95-9349|24.1255     |Yangon   |Credit card|
|765-26-6951|21.783 

In [33]:
sales_df2['Invoice ID','gross income','City','Payment'].orderBy('City','gross income',ascending=False).show(truncate=False)

+-----------+------------+------+-----------+
|Invoice ID |gross income|City  |Payment    |
+-----------+------------+------+-----------+
|687-47-8271|49.49       |Yangon|Credit card|
|325-77-6186|45.325      |Yangon|Ewallet    |
|384-59-6655|44.397      |Yangon|Cash       |
|704-48-3927|44.335      |Yangon|Ewallet    |
|827-77-7633|44.1405     |Yangon|Cash       |
|139-32-4183|43.866      |Yangon|Ewallet    |
|805-86-0265|42.282      |Yangon|Cash       |
|698-98-5964|40.605      |Yangon|Credit card|
|138-17-5109|40.1445     |Yangon|Credit card|
|638-60-7125|39.824      |Yangon|Credit card|
|232-11-3025|39.385      |Yangon|Cash       |
|652-43-6591|38.916      |Yangon|Credit card|
|880-46-5796|38.46       |Yangon|Ewallet    |
|746-94-0204|37.458      |Yangon|Credit card|
|453-33-6436|37.248      |Yangon|Cash       |
|243-55-8457|37.22       |Yangon|Ewallet    |
|416-17-9926|37.11       |Yangon|Credit card|
|884-80-6021|36.735      |Yangon|Ewallet    |
|742-04-5161|36.39       |Yangon|C

### Display a subset of rows
"where" Condition in SQL

In [34]:
sales_df2['Invoice ID','gross income','City','Payment'][sales_df2['City'] == 'Yangon'].show(truncate=False)

+-----------+------------+------+-----------+
|Invoice ID |gross income|City  |Payment    |
+-----------+------------+------+-----------+
|750-67-8428|26.1415     |Yangon|Ewallet    |
|631-41-3108|16.2155     |Yangon|Credit card|
|123-19-1176|23.288      |Yangon|Ewallet    |
|373-73-7910|30.2085     |Yangon|Ewallet    |
|355-53-5943|20.652      |Yangon|Ewallet    |
|665-32-9167|3.626       |Yangon|Credit card|
|365-64-0515|11.7375     |Yangon|Ewallet    |
|252-56-2699|21.595      |Yangon|Ewallet    |
|829-34-3910|35.69       |Yangon|Cash       |
|656-95-9349|24.1255     |Yangon|Credit card|
|765-26-6951|21.783      |Yangon|Credit card|
|329-62-1586|8.2005      |Yangon|Credit card|
|636-48-8204|8.64        |Yangon|Ewallet    |
|549-59-1358|13.2945     |Yangon|Ewallet    |
|227-03-5010|21.036      |Yangon|Credit card|
|189-17-4241|8.767       |Yangon|Credit card|
|848-62-7243|11.2005     |Yangon|Cash       |
|595-11-5460|9.658       |Yangon|Credit card|
|129-29-8530|15.655      |Yangon|E

#### AND | OR

In [35]:
sales_df2['Invoice ID','gross income','City','Payment'][(sales_df2['City'] == 'Yangon') & (sales_df2['Payment'] == 'Ewallet')].\
    show(truncate=False)

+-----------+------------+------+-------+
|Invoice ID |gross income|City  |Payment|
+-----------+------------+------+-------+
|750-67-8428|26.1415     |Yangon|Ewallet|
|123-19-1176|23.288      |Yangon|Ewallet|
|373-73-7910|30.2085     |Yangon|Ewallet|
|355-53-5943|20.652      |Yangon|Ewallet|
|365-64-0515|11.7375     |Yangon|Ewallet|
|252-56-2699|21.595      |Yangon|Ewallet|
|636-48-8204|8.64        |Yangon|Ewallet|
|549-59-1358|13.2945     |Yangon|Ewallet|
|129-29-8530|15.655      |Yangon|Ewallet|
|272-65-1806|27.396      |Yangon|Ewallet|
|635-40-6220|35.84       |Yangon|Ewallet|
|287-21-9091|33.6015     |Yangon|Ewallet|
|873-51-0671|7.693       |Yangon|Ewallet|
|594-34-4444|4.858       |Yangon|Ewallet|
|865-92-6136|7.9125      |Yangon|Ewallet|
|704-48-3927|44.335      |Yangon|Ewallet|
|645-44-1170|26.1315     |Yangon|Ewallet|
|642-32-2990|5.48        |Yangon|Ewallet|
|575-30-8091|29.0        |Yangon|Ewallet|
|239-10-7476|23.385      |Yangon|Ewallet|
+-----------+------------+------+-

In [36]:
sales_df2['Invoice ID','gross income','City','Payment'][(sales_df2['City'] == 'Yangon') | (sales_df2['Payment'] == 'Ewallet')].\
    show(truncate=False)

+-----------+------------+---------+-----------+
|Invoice ID |gross income|City     |Payment    |
+-----------+------------+---------+-----------+
|750-67-8428|26.1415     |Yangon   |Ewallet    |
|631-41-3108|16.2155     |Yangon   |Credit card|
|123-19-1176|23.288      |Yangon   |Ewallet    |
|373-73-7910|30.2085     |Yangon   |Ewallet    |
|699-14-3026|29.8865     |Naypyitaw|Ewallet    |
|355-53-5943|20.652      |Yangon   |Ewallet    |
|315-22-5665|36.78       |Naypyitaw|Ewallet    |
|665-32-9167|3.626       |Yangon   |Credit card|
|351-62-0822|2.896       |Mandalay |Ewallet    |
|365-64-0515|11.7375     |Yangon   |Ewallet    |
|252-56-2699|21.595      |Yangon   |Ewallet    |
|829-34-3910|35.69       |Yangon   |Cash       |
|656-95-9349|24.1255     |Yangon   |Credit card|
|765-26-6951|21.783      |Yangon   |Credit card|
|329-62-1586|8.2005      |Yangon   |Credit card|
|319-50-3348|4.03        |Mandalay |Ewallet    |
|300-71-4605|21.51       |Naypyitaw|Ewallet    |
|371-85-5789|13.197 

### Aggregation | Group By

In [37]:
sales_df2.groupBy('Payment','City').count().show()

+-----------+---------+-----+
|    Payment|     City|count|
+-----------+---------+-----+
|Credit card| Mandalay|  109|
|Credit card|Naypyitaw|   98|
|       Cash|Naypyitaw|  124|
|       Cash|   Yangon|  110|
|    Ewallet| Mandalay|  113|
|Credit card|   Yangon|  104|
|    Ewallet|Naypyitaw|  106|
|       Cash| Mandalay|  110|
|    Ewallet|   Yangon|  126|
+-----------+---------+-----+



In [38]:
sales_df2['Payment','City','Quantity'].groupBy('Payment','City').sum().show()

+-----------+---------+-------------+
|    Payment|     City|sum(Quantity)|
+-----------+---------+-------------+
|Credit card| Mandalay|          599|
|Credit card|Naypyitaw|          543|
|       Cash|Naypyitaw|          696|
|       Cash|   Yangon|          572|
|    Ewallet| Mandalay|          593|
|Credit card|   Yangon|          580|
|    Ewallet|Naypyitaw|          592|
|       Cash| Mandalay|          628|
|    Ewallet|   Yangon|          707|
+-----------+---------+-------------+



In [39]:
sales_df2['Payment','Quantity','gross income'].groupBy('Payment').mean().show()

+-----------+-----------------+------------------+
|    Payment|    avg(Quantity)| avg(gross income)|
+-----------+-----------------+------------------+
|    Ewallet|5.484057971014493|15.181933333333335|
|       Cash|5.511627906976744|15.532470930232577|
|Credit card|5.536977491961415| 15.42904180064309|
+-----------+-----------------+------------------+



In [40]:
# This is REALLY CLEVER
#
groupBy51 = ['Payment']
aggregate51 = ['gross income','Quantity']
#funs = [mean, sum, max]
funs = [count,mean, sum]

exprs51 = [f(col(c)) for f in funs for c in aggregate51]

sales_df2['Payment','Quantity','gross income'].groupBy(groupBy51).agg(*exprs51).show()

+-----------+-------------------+---------------+------------------+-----------------+-----------------+-------------+
|    Payment|count(gross income)|count(Quantity)| avg(gross income)|    avg(Quantity)|sum(gross income)|sum(Quantity)|
+-----------+-------------------+---------------+------------------+-----------------+-----------------+-------------+
|    Ewallet|                345|            345|15.181933333333335|5.484057971014493|5237.767000000001|         1892|
|       Cash|                344|            344|15.532470930232577|5.511627906976744|5343.170000000006|         1896|
|Credit card|                311|            311| 15.42904180064309|5.536977491961415|4798.432000000001|         1722|
+-----------+-------------------+---------------+------------------+-----------------+-----------------+-------------+



### SQL Table Operations

#### Dataframe to Temporary Table

In [41]:
sales_df2.createOrReplaceTempView("tSales")
spark39.sql("describe tSales").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Invoice ID|   string|   NULL|
|              Branch|   string|   NULL|
|                City|   string|   NULL|
|       Customer type|   string|   NULL|
|              Gender|   string|   NULL|
|        Product line|   string|   NULL|
|          Unit price|   double|   NULL|
|            Quantity|      int|   NULL|
|              Tax 5%|   double|   NULL|
|               Total|   double|   NULL|
|                Date|     date|   NULL|
|                Time|timestamp|   NULL|
|             Payment|   string|   NULL|
|                cogs|   double|   NULL|
|gross margin perc...|   double|   NULL|
|        gross income|   double|   NULL|
|              Rating|   double|   NULL|
+--------------------+---------+-------+



In [42]:
spark39.sql("show tables").show()
spark39.sql("select * from tsales limit 3").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   tsales|       true|
+---------+---------+-----------+

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-04-29 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|

#### SQL Based Analysis

In [43]:
# NOTE backtick ` used to enclose column names with spaces inside!
spark39.sql("select payment, count(*), sum(quantity), avg(quantity), sum(`gross income`), avg(`gross income`) from tsales group by payment ").show()

+-----------+--------+-------------+-----------------+-----------------+------------------+
|    payment|count(1)|sum(quantity)|    avg(quantity)|sum(gross income)| avg(gross income)|
+-----------+--------+-------------+-----------------+-----------------+------------------+
|    Ewallet|     345|         1892|5.484057971014493|5237.767000000001|15.181933333333335|
|       Cash|     344|         1896|5.511627906976744|5343.170000000006|15.532470930232577|
|Credit card|     311|         1722|5.536977491961415|4798.432000000001| 15.42904180064309|
+-----------+--------+-------------+-----------------+-----------------+------------------+



#### Temporary to Permanent Table

In [45]:
#sql_drop_table = """drop table if exists praxisdb.sales_T"""
#sql_drop_database = """drop database if exists praxisdb cascade"""
#sql_create_database = """create database if not exists praxisdb location '/content/praxisdb/'"""
#sql_create_table = """create table if not exists praxisdb.sales_T using parquet as select  * from tSales"""

spark39.sql("drop table if exists praxisdb.sales_T")
spark39.sql("drop database if exists praxisdb cascade")
spark39.sql("create database if not exists praxisdb location '/content/praxisdb/'")
spark39.sql("create table if not exists praxisdb.sales_T using parquet as select  * from tSales")

DataFrame[]

In [46]:
spark39.sql("use praxisdb")
spark39.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| praxisdb|  sales_t|      false|
|         |   tsales|       true|
+---------+---------+-----------+



In [ ]:
#spark39.sql("select `gross income` from praxisdb.sales_t").show()

In [47]:
spark39.sql("select * from praxisdb.sales_t limit 5").show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-04-29 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|2019-03-08|2024-04-29 10:29:00|       Cash|  76.4|            4.761904762| 

# NetFlix Data

In [48]:
!head NetFlix.csv

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,TV Show,3%,,"João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi & Fantasy","In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor."
s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies","After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive."
s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai, Mark Lee, Susan

## Dataframe Operations

In [49]:
netflix_df1 = spark39.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/NetFlix.csv")
#netflix_df1.printSchema()
#netflix_df1.show(10, truncate=False)

In [50]:
netflix_df1 = netflix_df1.withColumn('date_added',to_date(col('date_added'), "MMMM d, yyyy"))
netflix_df1.printSchema()
netflix_df1.show(10, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

+-------+-------+-----+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+----------+------------+------+---------+--------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### SQL Table

In [51]:
netflix_df1.createOrReplaceTempView("tNetFlix")
#spark39.sql("describe tNetFlix").show()
spark39.sql("select * from tnetflix limit 10").show(truncate = False)

+-------+-------+-----+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+----------+------------+------+---------+--------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|show_id|type   |title|director         |cast                                                                                                                                                                      |country      |date_added|release_year|rating|duration |listed_in                                               |description                                                                                                    

In [52]:
spark39.sql("select count(*) from tnetflix").show(truncate=False)

+--------+
|count(1)|
+--------+
|7789    |
+--------+



In [53]:
spark39.sql("select * from tnetflix where show_id in ( 's6109', 's6110','s4417', 's6658')").show(truncate=False)

+-------+-----+-------------------------------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+----------+------------+------+--------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|show_id|type |title                          |director                |cast                                                                                                                                                    |country       |date_added|release_year|rating|duration|listed_in      |description                                                                                                                                          |
+-------+-----+-------------------------------+------------------------+----------------------------------

In [54]:
# Why is this output not correct?
#

sql = 'select \
show_id, title, director, cast from tnetflix \
where cast = director'
spark39.sql(sql).show(truncate=False)

sql = 'select count(*) from tnetflix where cast = director'
spark39.sql(sql).show(truncate=False)


+-------+-----------------------------------------------------------------------+----------------------+----------------------+
|show_id|title                                                                  |director              |cast                  |
+-------+-----------------------------------------------------------------------+----------------------+----------------------+
|s170   |A Family Affair                                                        |Tom Fassaert          |Tom Fassaert          |
|s396   |Alien Contact: Outer Space                                             |J. Michael Long       |J. Michael Long       |
|s483   |Amy Schumer Growing                                                    |Amy Schumer           |Amy Schumer           |
|s484   |Amy Schumer: The Leather Special                                       |Amy Schumer           |Amy Schumer           |
|s649   |Aziz Ansari Live at Madison Square Garden                              |Aziz Ansari           |

In [55]:
spark39.sql("select title, director from tnetflix where director like '%,%'").show(truncate=False)

+------------------------------------+--------------------------------------------------+
|title                               |director                                          |
+------------------------------------+--------------------------------------------------+
|(T)ERROR                            |Lyric R. Cabral, David Felix Sutcliffe            |
|#AnneFrank - Parallel Stories       |Sabina Fedeli, Anna Migotto                       |
|14 Cameras                          |Scott Hussion, Seth Fuller                        |
|14 Minutes from Earth               |Adam Davis, Jerry Kolber, Trey Nelson, Erich Sturm|
|2 Alone in Paris                    |Ramzy Bedia, Éric Judor                           |
|21 & Over                           |Jon Lucas, Scott Moore                            |
|365 Days                            |Barbara Białowąs, Tomasz Mandes                   |
|48 Christmas Wishes                 |Marco Deufemia, Justin G. Dyck                    |
|7 Din Moh

In [56]:
sql = 'select count(*) from tnetflix where cast like "%"+director+"%"'
spark39.sql(sql).show(truncate=False)

+--------+
|count(1)|
+--------+
|0       |
+--------+



## RDD Operations

In [57]:
!sed '1d' NetFlix.csv > Headless_NetFlix.csv
netflix_RDD1 = sc.textFile("Headless_NetFlix.csv")
# Two Extra Lines here ... WHY? WHY?
netflix_RDD1.count()


7789

In [58]:
netflix_RDD1.sample(False,0.001,None).take(3)

['s1241,TV Show,Castlevania,,"Richard Armitage, James Callis, Graham McTavish, Alejandra Reynoso, Tony Amendola, Matt Frewer, Emily Swallow",United States,"March 5, 2020",2020,TV-MA,3 Seasons,"Anime Series, TV Horror, TV Thrillers",A vampire hunter fights to save a besieged city from an army of otherworldly beasts controlled by Dracula himself. Inspired by the classic video games.',
 's2868,Movie,Hunt to Kill,Keoni Waxman,"Steve Austin, Gil Bellows, Gary Daniels, Marie Avgeropoulos, Emilie Ullerup, Michael Eklund, Eric Roberts, Michael Hogan, Donnelly Rhodes, Adrian Holmes",Canada,"January 1, 2020",2010,R,97 min,Action & Adventure,U.S. Border Patrol agent and deadly hunter Jim Rhodes takes violent revenge when a group of desperate thieves kidnaps his daughter after a robbery.',
 's3011,TV Show,Insatiable,,"Dallas Roberts, Debby Ryan, Alyssa Milano, Christopher Gorham, Kimmy Shields, Michael Provost, Erinn Westbrook, Daniel Kang, Arden Myrin",United States,"October 11, 2019",2019,TV-MA,

In [59]:
def split_11(text):
    t1 = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '@', text)        # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
                                                                # commas within quotes are replaced by @
    t2 = re.sub(r'"', '', t1)                                   # quotes removed
    l = t2.split(",")
    #l4 = l[4]
    l3 = re.sub(r'@', ',', l[3])                                # @ restored to commas
    l4 = re.sub(r'@', ',', l[4])
    l5 = re.sub(r'@', ',', l[5])
    l6 = re.sub(r'@', ',', l[6])
    l10 = re.sub(r'@', ',', l[10])
    l11 = re.sub(r'@', ',', l[11])
    return(l[0],l[1],l[2],l3,l4,l5,l6,l[7],l[8],l[9],l10,l11)

In [60]:
# Parse Netflix data
#
netflix_RDD1.sample(False,0.001,None).map(lambda x: split_11(x)).take(3)
#netflix_RDD2

[('s392',
  'Movie',
  'Alibaba Aur 40 Chor',
  'Latif Faiziyev, Umesh Mehra',
  'Dharmendra, Hema Malini, Zeenat Aman, Rolan Bykov, Prem Chopra, Yakub Akhmedov, Madan Puri, Sofiko Chiaureli',
  'Soviet Union, India',
  'January 15, 2018',
  '1980',
  'TV-14',
  '147 min',
  'Action & Adventure, International Movies, Sci-Fi & Fantasy',
  'A simple village man is thrown into a web of political intrigue after he takes on a powerful and ruthless gang of bandits.'),
 ('s499',
  'Movie',
  'An Upper Egyptian',
  'Ismail Farouk, Hazem Fouda',
  'Mohamed Ramadan, Randa El Behery, Nermin Maher, Inas El Naggar, Mayar El Gheity, Hassan Abdulfattah, Sabri Abdulmonem, Alaa Zenhom, Abdalah Mishrif, Mohamed Soliman',
  'Egypt',
  'June 20, 2019',
  '2014',
  'TV-MA',
  '85 min',
  'Comedies, Dramas, International Movies',
  'Eager to settle down, Abdullah searches for the right woman to marry – but the one he truly longs for may not feel the same way.'),
 ('s887',
  'Movie',
  'Bhangra Paa Le',
  'S

In [61]:
netflix_RDD1.filter(lambda x: "Harry Elfont," in x).take(3)

['s1195,Movie,Can\'t Hardly Wait,"Harry Elfont, Deborah Kaplan","Jennifer Love Hewitt, Ethan Embry, Charlie Korsmo, Lauren Ambrose, Peter Facinelli, Seth Green, Michelle Brookhurst, Alexander Martin, Erik Palladino, Channon Roe, Sean Patrick Thomas",United States,"January 1, 2021",1998,PG-13,101 min,"Comedies, Romantic Movies","At a wild high school graduation party, aspiring writer Preston hopes to declare his love to popular girl Amanda, who just got dumped by the class jock."']

In [62]:
netflix_RDD1.filter(lambda x: "Jacques Perrin," in x).collect()

['s4561,Movie,Océans,"Jacques Perrin, Jacques Cluzaud","Pierce Brosnan, Jacques Perrin, Rie Miyazawa, Lancelot Perrin","France, Switzerland, Spain, United States, United Arab Emirates","September 19, 2019",2009,G,84 min,"Documentaries, International Movies",This documentary examines the vital role water plays in human existence and the cause-and-effect interplay between oceans and the environment.']

In [63]:
def split_4(text):
    t1 = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '@', text)        # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
                                                                # commas within quotes are replaced by @
    t2 = re.sub(r'"', '', t1)
    l = t2.split(",")
    l3 = re.sub(r'@', ',', l[3])                                # @ restored to commas
    l4 = re.sub(r'@', ',', l[4])
    l5 = re.sub(r'@', ',', l[5])
    return(l[0],l[1],l[2],l3,l4,l5)

netflix_RDD1.sample(False,0.02,None).map(lambda x: split_4(x)).take(3)

[('s84',
  'Movie',
  '20th Century Women',
  'Mike Mills',
  'Annette Bening, Elle Fanning, Greta Gerwig, Lucas Jade Zumann, Billy Crudup',
  'United States'),
 ('s90',
  'Movie',
  '24 Hours to Live',
  'Brian Smrz',
  'Ethan Hawke, Xu Qing, Paul Anderson, Rutger Hauer, Tyrone Keogh, Nathalie Boltt, Liam Cunningham',
  'South Africa, China, United States'),
 ('s113',
  'TV Show',
  '45 rpm',
  '',
  'Carlos Cuevas, Guiomar Puerta, Iván Marcos, Israel Elejalde, Eudald Font, Luis Larrodera, Carmen Gutiérrez, Diana Gómez, Vito Sanz, Marina San José, Pere Ponce, Joan Pera',
  'Spain')]

### Identify Movies where Director is part of Cast

In [64]:
# Filter rows where Director is part of the Cast
#
def fDirInCast(text):
    t1 = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '@', text)        # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
    t2 = re.sub(r'"', '', t1)
    l = t2.split(",")
    #if len(l) < 3:                                              # WHY DO WE NEED THIS? To eliminate badly formed lines
    #    return(False)
    #else:
    if (l[3] == '') or (l[4] == ''):                        # WHY DO WE NEED THIS?  To eliminate rows where dir or cast is blank
        return(False)
    else:
        dirs = set(l[3].split("@"))
        cast = set(l[4].split("@"))
        common = dirs.intersection(cast)
        if len(common) > 0:
            return(True)
        else:
            return(False)


#sales_wallet = sales_RDD1.filter(lambda x: ",Ewallet," in x)
#netflix_RDD5 = netflix_RDD1.filter(lambda x: x4m11F(x))
netflix_RDD1.sample(False,0.01,None).filter(lambda x: fDirInCast(x)).map(lambda x: split_4(x)).take(5)

[('s649',
  'Movie',
  'Aziz Ansari Live at Madison Square Garden',
  'Aziz Ansari',
  'Aziz Ansari',
  'United States'),
 ('s4281',
  'Movie',
  'Munafik 2',
  'Syamsul Yusof',
  'Syamsul Yusof, Maya Karin, Nasir Bilal Khan, Rahim Razali, Fizz Fairuz, Fauzi Nawawi, Mawi, Ku Faridah',
  'Malaysia')]

In [65]:
netflix_RDD1.sample(False,0.02,None).filter(lambda x: fDirInCast(x)).map(lambda x: split_4(x)).take(5)

[('s1091',
  'Movie',
  'Breaking Free',
  'Sridhar Rangayan',
  'Sridhar Rangayan',
  'India'),
 ('s1649',
  'Movie',
  'De Film van Dylan Haegens',
  'Dylan Haegens, Bas van Teylingen',
  'Dylan Haegens, Nick Golterman, Marit Brugman, Ilse Warringa, Teun Peters, Patrick Stoof, Rick Vermeulen, Rico Verhoeven',
  'Netherlands'),
 ('s3024',
  'Movie',
  'Instructions Not Included',
  'Eugenio Derbez',
  'Eugenio Derbez, Jessica Lindsey, Loreto Peralta, Daniel Raymont, Alessandra Rosaldo, Hugo Stiglitz, Arcelia Ramírez, Sammy Pérez, Agustín Bernal, Karla Souza',
  'Mexico'),
 ('s4417',
  'Movie',
  'Natalia Valdebenito: Gritona',
  'Natalia Valdebenito',
  'Natalia Valdebenito',
  'United States')]

In [66]:
#netflix_RDD1.filter(lambda x: fDirCast(x)).map(lambda x: split_4(x)).count()
netflix_RDD1.filter(lambda x: fDirInCast(x)).map(lambda x: split_4(x)).count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 81.0 failed 1 times, most recent failure: Lost task 1.0 in stage 81.0 (TID 101) (f8e23dc1ea46 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.10/d

#### Trapping the Bad Data

In [67]:
# Filter rows where Director is part of the Cast
#
def fDirInCast2(text):
    try:
        t1 = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '@', text)        # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
        t2 = re.sub(r'"', '', t1)
        l = t2.split(",")
        if (l[3] == '') or (l[4] == ''):                        # WHY DO WE NEED THIS?  To eliminate rows where dir or cast is blank
            return(False)
        else:
            dirs = set(l[3].split("@"))
            cast = set(l[4].split("@"))
            common = dirs.intersection(cast)
            if len(common) > 0:
                return(True)
            else:
                return(False)
    except:
        return(False)


#netflix_RDD1.sample(False,0.01,None).filter(lambda x: fDirInCast2(x)).map(lambda x: split_4(x)).take(5)
netflix_RDD1.filter(lambda x: fDirInCast2(x)).map(lambda x: split_4(x)).count()

176

In [68]:
# Filter rows where Director is part of the Cast
#
def fDirInCast2x(text):
    try:
        t1 = re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '@', text)        # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
        t2 = re.sub(r'"', '', t1)
        l = t2.split(",")
        if (l[3] == '') or (l[4] == ''):                        # WHY DO WE NEED THIS?  To eliminate rows where dir or cast is blank
            return(False)
        else:
            dirs = set(l[3].split("@"))
            cast = set(l[4].split("@"))
            common = dirs.intersection(cast)
            if len(common) > 0:
                return(False)
            else:
                return(False)
    except:
        print('Error in '+ text)
        return(True)

#netflix_RDD1.sample(False,0.01,None).filter(lambda x: fDirInCast2(x)).map(lambda x: split_4(x)).take(5)
netflix_RDD1.filter(lambda x: fDirInCast2x(x)).collect()

[' and probably will."', 's6658,Movie,"The Memphis Belle: A Story of a']

##### Error 1

In [ ]:
#netflix_RDD1.filter(lambda x: ' and probably will."' in x).collect()

In [69]:
# https://unix.stackexchange.com/questions/333121/how-to-find-lines-containing-a-string-and-then-printing-those-specific-lines-and
!grep -nr ' and probably will.' /content/NetFlix.csv

6111: and probably will."


In [70]:
# spotting the two extra lines
#
!grep -nr -A2 -B2 ' and probably will.' /content/NetFlix.csv

6109-s6108,Movie,The Bad Batch,Ana Lily Amirpour,"Suki Waterhouse, Jason Momoa, Keanu Reeves, Jim Carrey, Giovanni Ribisi, Yolonda Ross",United States,"September 22, 2017",2016,R,119 min,"Dramas, Independent Movies, Thrillers","Banished to a wasteland of undesirables, a young woman struggles to find her feet among a drug-soaked desert society and an enclave of cannibals."
6110-s6109,Movie,The Bad Education Movie,Elliot Hegarty,"Jack Whitehall, Joanna Scanlan, Iain Glen, Ethan Lawrence, Layton Williams, Kae Alexander, Weruche Opia, Nikki Runeckles, Charlie Wernham, Jack Binstead",United Kingdom,"December 15, 2018",2015,TV-MA,87 min,Comedies,"Britain's most ineffective but caring teacher takes his class to Cornwall for one last school trip, where anything could happen –
6111: and probably will."
6112-s6110,Movie,The Bad Kids,"Keith Fulton, Louis Pepe",,United States,"April 1, 2017",2016,TV-MA,101 min,Documentaries,"In this documentary, teachers at a Mojave Desert high school adhere to a 

In [ ]:
#!grep -nr -A2 -B2 ' and probably will.' /content/Headless_NetFlix.csv

##### Error 2

In [ ]:
#netflix_RDD1.filter(lambda x: 's6658' in x).collect()

In [71]:
!grep -nr -A2 -B2 's6658' /content/NetFlix.csv

6658-s6656,TV Show,The Mechanism,,"Selton Mello, Caroline Abras, Enrique Diaz, Antonio Saboia, Lee Taylor, Otto Jr., Leonardo Medeiros, Osvaldo Mil, Jonathan Haagensen, Giulio Lopes, Susana Ribeiro",Brazil,"May 10, 2019",2019,TV-MA,2 Seasons,"Crime TV Shows, International TV Shows, TV Dramas",A scandal erupts in Brazil during an investigation of alleged government corruption via oil and construction companies. Loosely inspired by true events.
6659-s6657,Movie,The Memory of Water,Matías Bize,"Elena Anaya, Benjamín Vicuña, Néstor Cantillana, Sergio Hernández, Silvia Marty, Etienne Bobenrieth, Antonia Zegers, Pablo Cerda","Chile, Spain, Argentina, Germany","February 15, 2017",2015,TV-MA,88 min,"Dramas, International Movies","When the accidental death of a couple's young son sends them grieving in opposite directions, they're forced to re-examine their life and marriage."
6660:s6658,Movie,"The Memphis Belle: A Story of a
6661-Flying Fortress",William Wyler,,United States,"March 31, 2017",1

# IMDB Data

In [72]:
!head /content/IMDBmovies.csv

title,genre,budget,runtime,year,month,revenue
Avatar,Action,237000000,162.0,2009,12,2787965087
Pirates of the Caribbean: At World's End,Adventure,300000000,169.0,2007,5,961000000
Spectre,Action,245000000,148.0,2015,10,880674609
The Dark Knight Rises,Action,250000000,165.0,2012,7,1084939099
John Carter,Action,260000000,132.0,2012,3,284139100
Spider-Man 3,Fantasy,258000000,139.0,2007,5,890871626
Tangled,Animation,260000000,100.0,2010,11,591794936
Avengers: Age of Ultron,Action,280000000,141.0,2015,4,1405403694
Harry Potter and the Half-Blood Prince,Adventure,250000000,153.0,2009,7,933959197


## Dataframe Operations

In [73]:
imdb_df1 = spark39.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/IMDBmovies.csv")
imdb_df1.printSchema()
imdb_df1.show(10, truncate=False)

root
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- runtime: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- revenue: long (nullable = true)

+----------------------------------------+---------+---------+-------+----+-----+----------+
|title                                   |genre    |budget   |runtime|year|month|revenue   |
+----------------------------------------+---------+---------+-------+----+-----+----------+
|Avatar                                  |Action   |237000000|162.0  |2009|12   |2787965087|
|Pirates of the Caribbean: At World's End|Adventure|300000000|169.0  |2007|5    |961000000 |
|Spectre                                 |Action   |245000000|148.0  |2015|10   |880674609 |
|The Dark Knight Rises                   |Action   |250000000|165.0  |2012|7    |1084939099|
|John Carter                             |Action   |260000000|132.0  |2012|3   

### SQL Table

In [74]:
imdb_df1.createOrReplaceTempView("timdb")
#spark39.sql("describe tNetFlix").show()
spark39.sql("select * from timdb limit 10").show(truncate = False)

+----------------------------------------+---------+---------+-------+----+-----+----------+
|title                                   |genre    |budget   |runtime|year|month|revenue   |
+----------------------------------------+---------+---------+-------+----+-----+----------+
|Avatar                                  |Action   |237000000|162.0  |2009|12   |2787965087|
|Pirates of the Caribbean: At World's End|Adventure|300000000|169.0  |2007|5    |961000000 |
|Spectre                                 |Action   |245000000|148.0  |2015|10   |880674609 |
|The Dark Knight Rises                   |Action   |250000000|165.0  |2012|7    |1084939099|
|John Carter                             |Action   |260000000|132.0  |2012|3    |284139100 |
|Spider-Man 3                            |Fantasy  |258000000|139.0  |2007|5    |890871626 |
|Tangled                                 |Animation|260000000|100.0  |2010|11   |591794936 |
|Avengers: Age of Ultron                 |Action   |280000000|141.0  |

# Netflix Data AND IMDB Data together

## Dataframe SQL  Joins

In [ ]:
spark39.sql("select count(distinct(title)) from tnetflix").show(truncate = False)

+---------------------+
|count(DISTINCT title)|
+---------------------+
|7787                 |
+---------------------+



In [ ]:
spark39.sql("select count(distinct(title)) from timdb").show(truncate = False)

+---------------------+
|count(DISTINCT title)|
+---------------------+
|2000                 |
+---------------------+



In [ ]:
spark39.sql("select count(*) from timdb, tnetflix where timdb.title = tnetflix.title").show(truncate = False)

+--------+
|count(1)|
+--------+
|357     |
+--------+



In [ ]:
spark39.sql("select timdb.title, timdb.year, timdb.month, tnetflix.date_added, tnetflix.release_year from timdb, tnetflix where timdb.title = tnetflix.title").show(truncate = False)

+-------------------------------------+----+-----+----------+------------+
|title                                |year|month|date_added|release_year|
+-------------------------------------+----+-----+----------+------------+
|9                                    |2009|8    |2017-11-16|2009        |
|21                                   |2008|3    |2020-01-01|2008        |
|Æon Flux                             |2005|11   |2018-02-01|2005        |
|16 Blocks                            |2006|3    |2019-11-01|2006        |
|17 Again                             |2009|3    |2021-01-01|2009        |
|28 Days                              |2000|4    |2020-09-30|2000        |
|3 Days to Kill                       |2014|2    |2020-12-01|2014        |
|30 Minutes or Less                   |2011|8    |2021-01-01|2011        |
|50 First Dates                       |2004|2    |2020-12-01|2004        |
|A Bridge Too Far                     |1977|6    |2020-07-01|1977        |
|A Thousand Words        

## Dataframe Native Joins

In [ ]:
# Inner join
imdb_df1.join(netflix_df1,netflix_df1.title ==  imdb_df1.title,"inner").show(truncate=False)

+-------------------------------------+---------+---------+-------+----+-----+---------+-------+-----+-------------------------------------+-----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------+----------+------------+------+--------+--------------------------------------------------------+-----------------------------------------------------------

In [ ]:
imdb_df1.join(netflix_df1,netflix_df1.title ==  imdb_df1.title,"inner").count()

357

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)